In [1]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        # logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        # print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

2024-06-03 00:01:36.165676: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-03 00:01:36.286982: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


# 데이터 불러오기 및 클래스 지정과 분할  
The numbers of samples in each class for the test set are distributed as 106 (normal), 24 (AOM), 13 (CSOM), 28 (Earwax), and 20 (Other)

In [2]:
import os

# 데이터셋의 루트 디렉토리
dataset_directory = '/home/jeonk636/ear_classification/eardrumDs'

# 클래스 이름 및 라벨을 매핑하기 위한 딕셔너리 생성
# normal:0, Aom:1, Chornic:2, Earwax:3, Others:4
label_map = {
    'Aom': 1,
    'Chornic': 2,
    'Earwax': 3,
    'Normal': 0,
    'OtitExterna': 4,
    'tympanoskleros': 4,
    'Earventulation': 4,
    'Foreign': 4,
    'PseduoMembran': 4
}

# 이미지 파일 경로 및 라벨을 저장할 리스트
image_paths = []
image_labels = []

# 각 폴더에 대해 이미지 파일 경로 및 해당 라벨을 리스트에 추가
for label_folder in os.listdir(dataset_directory):
    folder_path = os.path.join(dataset_directory, label_folder)
    
    for image_filename in os.listdir(folder_path):
        # 폴더 내 파일이 실제 파일인지 확인하고 '.ipynb_checkpoints' 폴더를 건너뛴다.
        full_path = os.path.join(folder_path, image_filename)
        if os.path.isfile(full_path) and '.ipynb_checkpoints' not in full_path:
            image_paths.append(full_path)
            image_labels.append(label_map[label_folder])


In [3]:
label_0_count = image_labels.count(0)
label_1_count = image_labels.count(1)
label_2_count = image_labels.count(2)
label_3_count = image_labels.count(3)
label_4_count = image_labels.count(4)

print('Normal:', label_0_count)
print('Aom:', label_1_count)
print('Chornic:', label_2_count)
print('Earwax:', label_3_count)
print('Others:', label_4_count)

Normal: 534
Aom: 119
Chornic: 63
Earwax: 140
Others: 99


In [4]:
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd

# 이미지 경로와 라벨을 Numpy 배열로 변환
image_paths = np.array(image_paths)
image_labels = np.array(image_labels)

# 데이터프레임으로 변환
data = pd.DataFrame({'image_path': image_paths, 'label': image_labels})

# 클래스별로 지정된 수의 샘플을 추출하여 테스트 세트로 사용
test_counts = {
    0: 106,  # Normal
    1: 24,   # AOM
    2: 13,   # CSOM
    3: 28,   # Earwax
    4: 20    # Other
}

test_data_list = []

for label, count in test_counts.items():
    class_data = data[data['label'] == label]
    if len(class_data) < count:
        raise ValueError(f"Not enough samples for class {label}. Needed {count}, but only {len(class_data)} available.")
    test_data_list.append(class_data.sample(n=count, random_state=42))

test_data = pd.concat(test_data_list)

# 나머지 데이터를 훈련 세트로 사용
train_data = data.drop(test_data.index)

# 훈련 세트와 테스트 세트로 나누기
x_train = train_data['image_path'].values
y_train = train_data['label'].values
x_test = test_data['image_path'].values
y_test = test_data['label'].values

# # train 8: validation 2 분할
# x_train, x_test, y_train, y_test = train_test_split(
#     image_paths, image_labels, stratify=image_labels, test_size=0.2, random_state=42)

In [ ]:
# print("Train set:", len(x_train))
# print("test set:", len(x_test))

In [5]:
# 결과 출력
print("Training set size:", len(x_train))
print("Test set size:", len(x_test))
print("Test set class distribution:\n", test_data['label'].value_counts())

Training set size: 764
Test set size: 191
Test set class distribution:
 label
0    106
3     28
1     24
4     20
2     13
Name: count, dtype: int64


# keras을 사용한 데이터 증강  
224 by 224  
Normal 클래스가 가장 많은 샘플 수를 가지고 있으므로, 이를 기준으로 AOM, CSOM, Earwax, Other 클래스을 각각 3배, 7배, 3배, 4배 증강  
증강을 하면 train set에 있는 total sample 수는 normal 428, Aom 380, Csom 400, Earwax 448, others 395으로 각 클래스가 비슷해짐

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 데이터 증강 파라미터 정의
data_gen_args = dict(
    rescale=1./255,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.7, 1.3],  # ±30% 밝기 조절
    width_shift_range=0.2,  # 좌우 이동 범위 20%
    height_shift_range=0.2,  # 상하 이동 범위 20%
    zoom_range=0.2,  # ±20% 확대/축소
    rotation_range=20  # ±20도 회전
)

# 데이터 증강 제네레이터 생성
datagen = ImageDataGenerator(**data_gen_args)

# 증강된 샘플 수를 지정
augment_counts = {
    1: 3,  # AOM
    2: 7,  # CSOM
    3: 3,  # Earwax
    4: 4   # Other
}

# 증강된 데이터를 저장할 리스트
augmented_data = []

for label, count in augment_counts.items():
    class_data = train_data[train_data['label'] == label]
    for i in range(count):
        for index, row in class_data.iterrows():
            img = tf.keras.preprocessing.image.load_img(row['image_path'], target_size=(224, 224))
            img_array = tf.keras.preprocessing.image.img_to_array(img)
            img_array = np.expand_dims(img_array, axis=0)
            augmented_iter = datagen.flow(img_array, batch_size=1)
            augmented_img = next(augmented_iter)[0].astype(np.uint8)
            augmented_data.append({'image_path': row['image_path'], 'label': row['label']})

# 원본 데이터와 증강 데이터를 결합
augmented_df = pd.DataFrame(augmented_data)
train_data = pd.concat([train_data, augmented_df])

In [7]:
# 데이터 확인
print(train_data['label'].value_counts())
print(augmented_df['label'].value_counts())

label
3    448
0    428
2    400
4    395
1    380
Name: count, dtype: int64
label
2    350
3    336
4    316
1    285
Name: count, dtype: int64
